# Tweets streaming - Consumer
Read Stream into dataframe, add timestamp, sentiment and price, save aggregated window

Version 11: kill all entries with '0' in polarity or subjectivity

In [0]:
#!pip install textblob
#!pip install pycountry
#!pip install bs4

In [0]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob
import re
import pycountry
from datetime import datetime
import requests
import pandas as pd
import json
import time
from bs4 import BeautifulSoup

In [0]:
# Funktions to split away the timestamp from the tweet message
@udf
def split_start(text):
  stamp = text[:-27]
  return stamp
@udf
def split_back(text):
  stamp = text[-21:-4]     # -4 because reading it without milliseconds 
  return stamp


In [0]:
# Additional cleaning and splitting of the incoming text
def preprocessing(lines):
    words = lines.select(explode(split(lines.value, "t_end")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    words = words.withColumn('tweet_txt', split_start('word'))
    words = words.withColumn('stamp', split_back('word'))
    return words
  

In [0]:
# Apply Textblob Sentiment Analysis
def polarity_detection(text):
    return TextBlob(text).sentiment.polarity

def text_classification(words):
    # polarity detection
    polarity_detection_udf = udf(polarity_detection, StringType())
    words = words.withColumn("polarity", polarity_detection_udf("tweet_txt"))
    return words


In [0]:
# Language Detection
def language_detection(text):
    try:
      iso_code = TextBlob(text).detect_language()
      language = pycountry.languages.get(alpha_2=iso_code)
      language_name = language.name
    except:
      language_name = 'no language detected'
    return language_name

def apply_language(words):
    # language detection
    language_detection_udf = udf(language_detection, StringType())
    words = words.withColumn("language", language_detection_udf("tweet_txt"))
    return words
  

In [0]:
# Funtcion to call the current bitcoint price
@udf
def get_actual_crypto_price(crypto):
  base_url = 'https://coinmarketcap.com'
  request = requests.get(base_url)
  soup = BeautifulSoup(request.content, 'html.parser')
  data = soup.find('script', id="__NEXT_DATA__", type="application/json")
  coins = {}

  coin_data = json.loads(data.contents[0])
  listings = coin_data['props']['initialState']['cryptocurrency']['listingLatest']['data']

  for i in listings:
    crypto_curr = i['name']
    if crypto_curr.lower() == crypto.lower():
      evaluate_price = i['quotes'][2]
      #print(evaluate_price)
      coins[str(i['id'])] = i['slug']
      coins['currency'] = evaluate_price['name']
      coins['actual_price'] = evaluate_price['price']
      coins['percentChange24h'] = evaluate_price['percentChange24h']
  return coins['actual_price']


In [0]:
# Create Spark session
spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

# Read the tweet data from socket
lines = spark.readStream.format("socket") \
        .option("host", "localhost") \
        .option("port", 9997) \
        .load()

# Preprocess the data
words = preprocessing(lines)

# Apply column with text classification to define polarity
words = text_classification(words)

# Add column with language
words = apply_language(words)
                         
# Add a column with the currency lable 'bitcoin'
words = words.withColumn("crypto", lit('bitcoin'))


In [0]:
# Filter all tweets in English
# words = words.filter(words.language == "English")

# Filter out all tweets with polarity '0'
words = words.filter(words.polarity != "0.0")


In [0]:
# Re-format timestamp from string to TimestampType
format = '%Y-%m-%dT%H%M%S'
time_udf = udf(lambda x: datetime.strptime(x, format), TimestampType())
words = words.withColumn('t_stamp', time_udf('stamp'))

# Re-format polarity to float
words = words.withColumn('polarity', col('polarity').cast('float'))


In [0]:
# Create a 6 second window as basis for all subsequent analysis (in version 11 with watermark)
windowedStream = words.withWatermark('t_stamp', '6 seconds') \
                      .groupBy(window('t_stamp', '6 seconds', '6 seconds'))

# previous version without watermark:
# windowedStream = words.groupBy(window('t_stamp', '6 seconds', '6 seconds'))

# Aggregate the figures we need and add the bitcoin price
aggregationsStream = windowedStream \
        .agg(count('tweet_txt').alias('count_tweets') \
           , avg('polarity').alias('pol_avg') \
           , get_actual_crypto_price(first(col('crypto'))).cast('float').alias('window_price') \
           , max('t_stamp').alias('t_stamp')
           )


In [0]:
# display(words)
display(aggregationsStream)

window,count_tweets,pol_avg,window_price,t_stamp
"List(2021-06-12T14:05:18.000+0000, 2021-06-12T14:05:24.000+0000)",15,0.3463975489139557,35885.61,2021-06-12T14:05:23.000+0000
"List(2021-06-12T14:04:00.000+0000, 2021-06-12T14:04:06.000+0000)",3,0.27407407760620117,35885.61,2021-06-12T14:04:05.000+0000
"List(2021-06-12T14:04:06.000+0000, 2021-06-12T14:04:12.000+0000)",10,0.2086231058463454,35885.61,2021-06-12T14:04:11.000+0000
"List(2021-06-12T14:05:42.000+0000, 2021-06-12T14:05:48.000+0000)",11,0.2166499404067343,35885.61,2021-06-12T14:05:47.000+0000
"List(2021-06-12T14:04:24.000+0000, 2021-06-12T14:04:30.000+0000)",19,0.21390273225934883,35885.61,2021-06-12T14:04:29.000+0000
"List(2021-06-12T14:04:42.000+0000, 2021-06-12T14:04:48.000+0000)",14,0.1509498693048954,35885.61,2021-06-12T14:04:47.000+0000
"List(2021-06-12T14:04:30.000+0000, 2021-06-12T14:04:36.000+0000)",21,0.13094151441362642,35885.61,2021-06-12T14:04:35.000+0000
"List(2021-06-12T14:04:36.000+0000, 2021-06-12T14:04:42.000+0000)",15,0.13390674864252408,35885.61,2021-06-12T14:04:41.000+0000
"List(2021-06-12T14:05:36.000+0000, 2021-06-12T14:05:42.000+0000)",9,0.14262566301557753,35885.61,2021-06-12T14:05:41.000+0000
"List(2021-06-12T14:05:12.000+0000, 2021-06-12T14:05:18.000+0000)",17,0.18236779679051218,35885.61,2021-06-12T14:05:17.000+0000


In [0]:
'''
# Write the streaming DataFrame to a table
aggregationsStream.writeStream \
    .option("checkpointLocation", "path/to/checkpoint/dir") \
    .toTable("myTable")

spark.read.table("myTable").show()
'''

In [0]:
# Try to write to memory with 'append mode' (with watermark included in command 9)
streamingETLQuery = aggregationsStream \
  .writeStream \
  .format("memory") \
  .queryName("aggDF") \
  .outputMode("append")\
  .start()


In [0]:
spark.sql("select count(*) from aggDF").show()

+--------+
count(1)|
+--------+
 49|
+--------+

In [0]:
# Persistently store the result in a parquet file

spark.sql("select * from aggDF").write.parquet('dbfs:/FileStore/original/tweet_sparksql')

In [0]:
#type(streamingETLQuery)
spark.sql("select * from aggDF").sort('t_stamp').show() # interactively query in-memory table
#spark.sql("select count(*) from aggDF").show()

+--------------------+------------+-------------------+------------+-------------------+
 window|count_tweets| pol_avg|window_price| t_stamp|
+--------------------+------------+-------------------+------------+-------------------+
{2021-06-12 14:08...| 2| 0.1931818202137947| 35746.285|2021-06-12 14:08:47|
{2021-06-12 14:08...| 19| 0.343627726119992| 35746.285|2021-06-12 14:08:53|
{2021-06-12 14:08...| 9|0.26952861580583787| 35746.285|2021-06-12 14:08:58|
{2021-06-12 14:09...| 17|0.18173848815700588| 35746.285|2021-06-12 14:09:05|
{2021-06-12 14:09...| 13| 0.2009745127019974| 35746.285|2021-06-12 14:09:11|
{2021-06-12 14:09...| 12| 0.1960606093828877| 35746.285|2021-06-12 14:09:17|
{2021-06-12 14:09...| 19|0.22310179373935649| 35746.285|2021-06-12 14:09:23|
{2021-06-12 14:09...| 11|0.11306129531426863| 35746.285|2021-06-12 14:09:29|
{2021-06-12 14:09...| 15| 0.150386360536019| 35746.285|2021-06-12 14:09:35|
{2021-06-12 14:09...| 13| 0.1960303022597845| 35746.285|2021-06-12 14:09:41|
{2021-06-12 14:09...| 10| 0.2922424253076315| 35746.285|2021-06-12 14:09:46|
{2021-06-12 14:09...| 17| 0.3123870002215399| 35746.285|2021-06-12 14:09:53|
{2021-06-12 14:09...| 16|0.15396430692635477| 35746.285|2021-06-12 14:09:59|
{2021-06-12 14:10...| 11|0.13213966431265528| 35746.285|2021-06-12 14:10:05|
{2021-06-12 14:10...| 26| 0.2619919673038217| 35746.285|2021-06-12 14:10:11|
{2021-06-12 14:10...| 16|0.28656841767951846| 35746.285|2021-06-12 14:10:17|
{2021-06-12 14:10...| 17| 0.1816161835894865| 35746.285|2021-06-12 14:10:23|
{2021-06-12 14:10...| 11|0.11863005127419125| 35746.285|2021-06-12 14:10:29|
{2021-06-12 14:10...| 11|0.16925533657724207| 35746.285|2021-06-12 14:10:35|
{2021-06-12 14:10...| 14| 0.2738629953403558| 35746.285|2021-06-12 14:10:41|
+--------------------+------------+-------------------+------------+-------------------+
only showing top 20 rows

In [0]:
#%fs rm -r dbfs:/FileStore/original

res0: Boolean = true

In [0]:
#%fs ls dbfs:/FileStore/original/tweet_sparksql/